In [ ]:
!pip install cryptography ipywidgets > /dev/null


In [ ]:
import json
import os
import re
from cryptography.fernet import Fernet
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

DATA_FILE = 'passwords.json'
KEY_FILE = 'secret.key'

COLORS = {
    "card": "#ffffff",
    "input": "#e3eaf0",
    "text": "#ffffff",
    "border": "#b0c4de"
}

def generate_key():
    with open(KEY_FILE, 'wb') as f:
        f.write(Fernet.generate_key())

def load_key():
    if not os.path.exists(KEY_FILE):
        generate_key()
    return open(KEY_FILE, 'rb').read()

fernet = Fernet(load_key())

def save_password(website, username, password):
    encrypted = fernet.encrypt(password.encode()).decode()
    data = {}
    if os.path.exists(DATA_FILE):
        with open(DATA_FILE, 'r') as f:
            data = json.load(f)
    data[website] = {"username": username, "password": encrypted}
    with open(DATA_FILE, 'w') as f:
        json.dump(data, f, indent=4)

def search_by_name(name):
    if not os.path.exists(DATA_FILE):
        return []
    with open(DATA_FILE, 'r') as f:
        data = json.load(f)
    return [site for site, creds in data.items() if creds['username'].lower() == name.lower()]

def delete_by_name(name):
    if not os.path.exists(DATA_FILE):
        return 0
    with open(DATA_FILE, 'r') as f:
        data = json.load(f)
    total_before = len(data)
    data = {site: creds for site, creds in data.items() if creds['username'].lower() != name.lower()}
    with open(DATA_FILE, 'w') as f:
        json.dump(data, f, indent=4)
    return total_before - len(data)

def password_strength(password):
    if len(password) < 6:
        return "Very Weak"
    score = 0
    if re.search(r"[a-z]", password): score += 1
    if re.search(r"[A-Z]", password): score += 1
    if re.search(r"[0-9]", password): score += 1
    if re.search(r"[^A-Za-z0-9]", password): score += 1
    if len(password) >= 12: score += 1
    if score <= 1: return "Weak"
    elif score == 2: return "Medium"
    elif score == 3: return "Strong"
    else: return "Very Strong"

active_username = {"value": ""}

style = f"""
<style>
html, body {{
  height: 100%;
  margin: 0;
  background: linear-gradient(to top, #cde5ff, #004080);
  font-family: 'Segoe UI', sans-serif;
  display: flex;
  justify-content: center;
  align-items: center;
}}

.widget-label {{
  color: {COLORS['text']}; font-weight: bold;
}}

.widget-text input, .widget-password input {{
  background: {COLORS['input']}; color: #2c3e50; border: 1px solid {COLORS['border']};
  border-radius: 5px; padding: 5px;
}}

.widget-button {{
  background: {COLORS['card']}; color: #004080; border: 1px solid {COLORS['border']};
  border-radius: 5px; padding: 5px 10px;
  transition: all 0.3s ease-in-out;
}}

.widget-button:hover {{
  background-color: #f0f8ff;
  box-shadow: 0 0 10px #b0c4de;
  cursor: pointer;
}}

.output_wrapper, .output {{
  background: transparent; color: {COLORS['text']};
}}

h2, h4, p {{
  display: none;
}}

.center-box {{
  background-color: rgba(255,255,255,0.1);
  padding: 20px;
  border-radius: 15px;
  box-shadow: 0 8px 24px rgba(0,0,0,0.2);
  animation: fadeInUp 1s ease-out;
}}

@keyframes fadeInUp {{
  from {{
    opacity: 0;
    transform: translateY(30px);
  }}
  to {{
    opacity: 1;
    transform: translateY(0);
  }}
}}
</style>
"""
display(HTML(style))

name_input = widgets.Text(placeholder="👤 Your Name")
name_button = widgets.Button(description="Set Name")
name_output = widgets.Output()

def on_set_name(b):
    with name_output:
        clear_output()
        name = name_input.value.strip()
        if name:
            active_username["value"] = name
            print(f"Name set to: {name}")
        else:
            print("Please enter a name.")

name_button.on_click(on_set_name)

website_input = widgets.Text(placeholder="🌐 Website")
password_input = widgets.Password(placeholder="🔑 Password")
strength_label = widgets.Label(value="")
save_button = widgets.Button(description="Save Password")
save_output = widgets.Output()

def update_strength(change):
    pwd = password_input.value
    if pwd:
        strength_label.value = f"Password Strength: {password_strength(pwd)}"
    else:
        strength_label.value = ""

password_input.observe(update_strength, names='value')

def on_save(b):
    with save_output:
        clear_output()
        name = active_username["value"]
        website = website_input.value.strip()
        password = password_input.value.strip()
        if not (name and website and password):
            print("Fill all fields and set your name.")
            return
        save_password(website, name, password)
        print(f"Saved for {website} under '{name}'")

save_button.on_click(on_save)

search_button = widgets.Button(description="Show My Websites")
search_output = widgets.Output()

def on_search(b):
    with search_output:
        clear_output()
        name = active_username["value"]
        if not name:
            print("Set your name first.")
            return
        results = search_by_name(name)
        if results:
            print(f"Websites for '{name}':")
            for site in results:
                print(f"• {site}")
        else:
            print("No websites found.")

search_button.on_click(on_search)

delete_button = widgets.Button(description="Delete All for Name", button_style='danger')
delete_output = widgets.Output()

def on_delete(b):
    with delete_output:
        clear_output()
        name = active_username["value"]
        if not name:
            print("Set your name first.")
            return
        count = delete_by_name(name)
        if count > 0:
            print(f"Deleted {count} entries.")
        else:
            print("No entries found to delete.")

delete_button.on_click(on_delete)

main_box = widgets.VBox([
    widgets.HBox([name_input, name_button]),
    name_output,
    widgets.HTML("<br><b>Save Password</b>"),
    website_input, password_input, strength_label, save_button, save_output,
    widgets.HTML("<br><b>Search My Websites</b>"),
    search_button, search_output,
    widgets.HTML("<br><b>Delete All My Passwords</b>"),
    delete_button, delete_output
])

display(widgets.Box([widgets.HTML('<div class="center-box"></div>'), main_box],
        layout=widgets.Layout(justify_content='center', align_items='center')))
